In [3]:
import os
import cv2
import pickle
import imageio
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
from pprint import pprint

In [4]:
from enum import Enum
from collections import namedtuple

class ModelType(Enum):
    CLASSIFICATION = 0
    SEGMENTATION = 1

classlabels_viz_colors = ['black', 'green', 'yellow', 'blue', 'red', 'magenta', 'cyan',
                          'lightseagreen', 'brown', 'magenta', 'olive', 'wheat', 'white', 'black']
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 100]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

confidence_heatmap_viz_colors = ['black', 'blue', 'red', 'orange', 'yellow', 'lightgreen', 'lightseagreen']
confidence_heatmap_viz_bounds = [-1, 0,0.5,0.6,0.7,0.8,0.9,1]
confidence_heatmap_viz_cmap = mpl.colors.ListedColormap(confidence_heatmap_viz_colors)
confidence_heatmap_viz_norm = mpl.colors.BoundaryNorm(confidence_heatmap_viz_bounds, confidence_heatmap_viz_cmap.N)


LabelColor = namedtuple('LabelColor', ['name', 'id', 'trainid', 'color', 'category'])

LABEL_COLORS = [
    LabelColor('class1', 1, 0, (128, 0, 128), 'driveableterrain'),
    LabelColor('class2', 2, 1, (255, 0, 0), 'non-driveableterrain'),
    LabelColor('class3', 3, 2, (0, 0, 255), 'sky'),
    LabelColor('class4', 4, 3, (0, 255, 0), 'trees'),
    LabelColor('class5', 5, 4, (255, 0, 255), 'implement'),
    LabelColor('class6', 6, 5, (255, 255, 0), 'basket markers')
]

LABEL_COLORS_4CLASS = LABEL_COLORS[0:4]
LABEL_COLORS_5CLASS = LABEL_COLORS[0:5]
LABEL_COLORS_6CLASS = LABEL_COLORS[0:6]
LABEL_COLORS_SKY_DET = [LABEL_COLORS[0], LABEL_COLORS[2]]

LABEL_COLORS_IMPL = [
    LabelColor('class1', 1, 1, (128, 0, 128), 'implement'),
    LabelColor('class2', 2, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 3, 3, (0, 0, 255), 'harrow_tine'),
    LabelColor('class4', 4, 4, (0, 255, 0), 'basket'),
    LabelColor('class5', 5, 5, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

LABEL_COLORS_IMPL_REDUCED = [
    LabelColor('class0', 1, 0, (0, 0, 0), 'background'),
    LabelColor('class1', 2, 1, (0, 255, 0), 'implement'),
    LabelColor('class2', 3, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 4, 3, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

PLUG_LABEL_MAP ={0: 'no-plug', 1: 'plug'}
IMPL_SEGMENT_LABEL_MAP = {0: 'background', 1: 'implement', 2: 'sweep', 3:'basket_marker'}

In [5]:
def create_mpl_viz_outputs(output_path,
                           image,
                           prediction_labels,
                           confidences,
                           depth_img, 
                           image_title='Image', 
                           pred_title='Prediction', 
                           conf_title='Confidence', 
                           depth_title='Depth',
                           bbox_coords=[]):
    """
    Utility function to plot results based on order of input provided.

    Axes index can have different values based on input provided.

    For example: If image, prediction and groundtruth_label is provided, A three axes plot will be generated with
    image(ax1), ground_truth(ax2), prediction(ax3).

    Order of plot if all the inputs are provided will be in same order as arguments listed above.
    """
    axis_index = list(range(len(list(filter(lambda x: x is not None, [image,
                                                                      prediction_labels, confidences,
                                                                      depth_img])))))
    axis_curr_index = 0
    fig, axes = mpl.pyplot.subplots(1, len(axis_index), figsize=((60, 30)))
    if bbox_coords:
#         xmin, ymin, xmax, ymax = bbox_coords
        xmin, xmax, ymin, ymax = bbox_coords
        if (xmin < 0) or (ymin < 0):
            raise ValueError(f'Either {xmin} or {ymin} are negative')
        else:
            rect = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
                                     facecolor='none')
    else:
        rect = None

    if axis_curr_index < len(axes):
        # Grab only RGB channels from image, otherwise depth with distort the image when it is displayed
        axes[axis_curr_index].imshow(image)
        axes[axis_curr_index].set_title(image_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(depth_img, cmap='turbo')
        axes[axis_curr_index].set_title(depth_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(prediction_labels, classlabels_viz_cmap, classlabels_viz_norm, interpolation='nearest')
        if rect is not None:
            rect1 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',facecolor='none')
            axes[axis_curr_index].add_patch(rect1)
        axes[axis_curr_index].set_title(pred_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        c = np.max(confidences, axis=2)
        axes[axis_curr_index].imshow(c, confidence_heatmap_viz_cmap, confidence_heatmap_viz_norm, interpolation='nearest')
#         if rect is not None:
#             rect2 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
#                                           facecolor='none')
#             axes[axis_curr_index].add_patch(rect2)
        axes[axis_curr_index].set_title(conf_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    mpl.pyplot.savefig(output_path, pad_inches=0, bbox_inches='tight', dpi=150)
    mpl.pyplot.close('all')

def read_image(image_path):
    image = (np.load(image_path) * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def normalize_and_clip_depth(depth, max_depth):
    """
    Return an optionally normalized (and clipped) depth.
    """
    depth[np.isnan(depth)] = max_depth
    depth[depth > max_depth] = max_depth
    depth = ((depth) / max_depth).astype(np.float32)
    return depth

DEFAULT_TONEMAP_PARAMS = {"policy": "tonemap", "alpha": 0.25, "beta": 0.9, "gamma": 0.9, "eps": 1e-6}
def normalize_image(image, hdr_mode=True, normalization_params=DEFAULT_TONEMAP_PARAMS, return_8_bit=False):
    """
    Normalize an 8 bit image according to the specified policy.
    If return_8_bit, this returns an np.uint8 image, otherwise it returns a floating point
    image with values in [0, 1].
    """
    normalization_policy = normalization_params['policy']
    lower_bound = 0
    upper_bound = 1
    if np.isnan(hdr_mode):
        hdr_mode = False

    if hdr_mode and image.dtype == np.uint8:
        # The image was normalized during pack-perception (tonemap)
        if return_8_bit:
            return image
        lower_bound = 0.0
        upper_bound = 255.0
    elif normalization_policy == "percentile" and hdr_mode:
        lower_bound = np.array([np.percentile(image[..., i],
                                              normalization_params['lower_bound'],
                                              interpolation='lower')
                                for i in range(3)])
        upper_bound = np.array([np.percentile(image[..., i],
                                              normalization_params['upper_bound'],
                                              interpolation='lower')
                                for i in range(3)])
    elif normalization_policy == "percentile_vpu" and hdr_mode:
        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        brightness = (3 * r + b + 4 * g) / 8
        lower_bound = np.percentile(brightness, normalization_params['lower_bound'],
                                    interpolation='lower')
        upper_bound = np.percentile(brightness, normalization_params['upper_bound'],
                                    interpolation='lower')
    elif normalization_policy == "3sigma" and hdr_mode:
        sigma_size = normalization_params['sigma_size']
        min_variance = normalization_params['min_variance']
        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        brightness = (3 * r + b + 4 * g) / 8
        mean, sigma = np.mean(brightness), np.std(brightness)
        brightness_min, brightness_max = np.min(brightness), np.max(brightness)
        if (sigma * sigma_size) > mean:
            lmin = brightness_min
            lmax = min(brightness_max, mean * sigma_size)
            if (lmax - lmin) < min_variance:
                lmax = lmin + min_variance
            lower_bound = lmin
            upper_bound = lmax
        else:
            mean_var = mean - sigma_size * sigma
            output_min = max(brightness_min, mean_var)
            mean_var = mean + sigma_size * sigma
            output_max = min(brightness_max, mean_var)
            if (output_max - output_min) < min_variance:
                output_min = mean - min_variance / 2.0
                output_min = 0 if output_min < 0 else output_min
                output_max = output_min + min_variance
            lower_bound = output_min
            upper_bound = output_max
    elif normalization_policy == 'tonemap' and hdr_mode:
        if image.dtype != np.float32 and image.dtype != np.uint32:
            raise ValueError('HDR image type is {} instead of float32 or uint32'.format(image.dtype))
        alpha = normalization_params.get('alpha', DEFAULT_TONEMAP_PARAMS['alpha'])
        beta = normalization_params.get('beta', DEFAULT_TONEMAP_PARAMS['beta'])
        gamma = normalization_params.get('gamma', DEFAULT_TONEMAP_PARAMS['gamma'])
        eps = normalization_params.get('eps', DEFAULT_TONEMAP_PARAMS['eps'])

        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        lum_in = 0.2126 * r + 0.7152 * g + 0.0722 * b
        lum_norm = np.exp(gamma * np.mean(np.log(lum_in + eps)))
        c = alpha * lum_in / lum_norm
        c_max = beta * np.max(c)
        lum_out = c / (1 + c) * (1 + c / (c_max ** 2))
        image = image * (lum_out / (lum_in + eps))[..., None]
    elif normalization_policy == "none" and hdr_mode:
        lower_bound = 0.0
        upper_bound = 2**20 - 1
    elif normalization_policy == "default" or not hdr_mode:
        assert np.max(image) <= 255 and np.min(image) >= 0, "Image with default " \
            "mode should be in range [0,255]"
        lower_bound = 0.0
        upper_bound = 255.0
    else:
        raise ValueError(
            f"--normalization-policy '{normalization_policy}' is not supported! "
            f"(on image with hdr_mode={hdr_mode})")

    image = (image.astype(np.float32, copy=False) - lower_bound) / (upper_bound - lower_bound)

    if return_8_bit:
        image = np.clip(image * 255.0, 0.0, 255.0)
        image = np.uint8(image)
    else:
        image = np.clip(image, 0.0, 1.0)

    return image

def read_saved_frame(pred_dir, image_id):
    states_to_save = ['', 'false_positive', 'false_negative', 'large_object_false_negative', 'true_positive', 'true_negative']
    frame = None
    for state in states_to_save:
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.png')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.png'))
            break
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.jpg')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.jpg'))
            break
    return frame

def read_images(pred_dir, _id):
    if not os.path.isfile(os.path.join(pred_dir, _id+'_image.npy')):
        return None, None, None, None
    image = np.load(os.path.join(pred_dir, _id+'_image.npy'))
    
    # 100m capped depth
    depth = np.load(os.path.join(pred_dir, _id+'_depth.npy'))
#     # raw depth
#     raw_depth_dir = '/raum_raid/li.yu/data/Jupiter_rock_demo_2021/Jupiter_rock_demo_loamy06_Oct20_2021/model_processed_v4.1_sky_2e-3_lr_1e-3_color_aug_full_model_LR_consistency_regularization_0.2_epoch_23/images/'
#     stereo_data = np.load(os.path.join(raw_depth_dir, _id, 'stereo_output.npz'))
#     depth = stereo_data['point_cloud'][:,:,-1]
#     depth = normalize_and_clip_depth(depth, 200)
    
    pred_label = np.load(os.path.join(pred_dir, _id+'_pred_label.npy'))
    confidence = np.load(os.path.join(pred_dir, _id+'_confidence.npy'))
    return image, depth, pred_label, confidence

# def create_frame(pred_dir, pred_merged_dir, _id, recreate=False):
#     canvas_path = os.path.join(pred_merged_dir, _id+'.png')
#     if recreate:
#         image, depth, pred_label, confidence = read_images(pred_dir, _id)
#         if image is None:
#             return None
#         create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth)
#     frame = cv2.imread(canvas_path)
#     return frame

def get_bbox_coords(i=-1, bbox_range_list=[], bbox_coord_list=[]):
    for bi in range(len(bbox_range_list)):
        bbox_range = bbox_range_list[bi]
        if bbox_range[0] <= i <= bbox_range[1]:
            return bbox_coord_list[bi]
    return []

def process_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
    return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    canvas_path = os.path.join(pred_merged_dir, _id+'.png')
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return None, None, None, None, None, None, None, None
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
        create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, image_title=image_title, bbox_coords=bbox_coords)
    frame = cv2.imread(canvas_path)
    return frame, image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_diff_frame(pred_merged_dir, _id, image1, depth1, pred_label1, confidence1, 
                      image2, depth2, pred_label2, confidence2, pred_title='prediction', conf_title='confidence'):
    canvas_path = os.path.join(pred_merged_dir, _id+'_diff.png')
    image = np.abs(image1 - image2)
    depth = np.abs(depth1 - depth2)
    pred_label = np.abs(pred_label1 - pred_label2)
    confidence = np.abs(confidence1 - confidence2)
    create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, conf_title=conf_title)
    frame = cv2.imread(canvas_path)
    return frame

def read_raw_image_by_id(data_dir, _id):
    image_path = os.path.join(data_dir, 'images', _id, 'artifact_debayeredrgb_0_'+_id+'.png')
    image = cv2.imread(image_path)
    return image

def read_raw_image_by_row(data_dir, row):
    image = cv2.imread(os.path.join(data_dir, row.artifact_debayeredrgb_0_save_path))
    return image

def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()
    
from datetime import datetime, timedelta

def get_sequences(df, interval=5*60, per_camera=False):
    df = df.sort_values('collected_on')
    df['datetime'] = df.collected_on.apply(datetime.fromisoformat)
    sequence_dfs = []
    delta = timedelta(seconds=interval)
    start = True
    i0, i = 0, 0
    while i < len(df):
        if start:
            t0 = df.iloc[i].datetime
            start = False
        else:
            t1 = df.iloc[i].datetime
            if t1 - t0 > delta or i == len(df) - 1:
                chunk_df = df.iloc[i0 : i if i < len(df) - 1 else len(df)]
                if per_camera:
                    camera_locations = chunk_df.camera_location.unique()
                    camera_locations.sort()
                    for camera_location in camera_locations:
                        sequence_df = chunk_df[chunk_df.camera_location == camera_location]
                        sequence_df = sequence_df.sort_values('collected_on')
                        sequence_dfs.append(sequence_df)
                else:
                    sequence_dfs.append(chunk_df)
                start = True
                i0 = i
            else:
                t0 = t1
        i += 1
    return sequence_dfs

### Create video from raw left images

In [7]:
# create video from raw left images
# data_dir = '/data/jupiter/datasets/halo_failure_case_of_box_in_dust'
# data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images'
data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images_all_camera_sequence'
df = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))
# df = df[df.camera_location == 'rear-left']
df = df.sort_values('collected_on')

# ids = df.id.to_list()
# video_name = os.path.join(data_dir, f'video.mp4')
# create_video(ids, data_dir, video_name, read_raw_image_by_id, fps=1)

sub_dfs = get_sequences(df)
print(df.shape, len(sub_dfs))

# for i,sub_df in enumerate(sub_dfs):
#     ids = sub_df.id.to_list()
#     video_name = os.path.join(data_dir, f'dust_seq{i}.mp4')
#     create_video(ids, data_dir, video_name, read_raw_image_by_id, fps=3)

(1332, 171) 15


In [8]:
seq_ids = [13, 0, 5, 7, 9]


In [10]:
concat_dfs = [concat_dfs[-1]] + concat_dfs[:4]
cam_dfs = [pd.concat([concat_dfs[i][j] for i in range(len(concat_dfs))]) for j in range(4)]
min_len = min(len(cdf) for cdf in cam_dfs)
print([len(cdf) for cdf in cam_dfs])

[24, 24, 24, 24]


In [11]:
# read from multiple cameras and put in once frame
# cameras = ['T01', 'T02', 'T03', 'T04']
cameras = ['T02', 'T06', 'T14', 'T10']
# cameras = [f'T{str(i+1).zfill(2)}' for i in range(16)]
print(cameras)
concat_dfs = []
for i,sub_df in enumerate(sub_dfs):
    # cam_dfs = [sub_df[sub_df.camera_location == c] for c in cameras]
    # min_len = min(len(cdf) for cdf in cam_dfs)
    # cam_dfs = [cdf.sort_values('collected_on').iloc[:min_len] for cdf in cam_dfs]
    if min_len < 2:
        continue
    print(i, [len(cdf) for cdf in cam_dfs])
    # print(i, [cdf.iloc[0].collected_on for cdf in cam_dfs])
    
    if i in seq_ids:
        if i == 13:
            concat_dfs.append([cdf.sort_values('collected_on').iloc[:min_len//3*2] for cdf in cam_dfs])
        else:
            concat_dfs.append(cam_dfs)

    frame = read_raw_image_by_row(data_dir, sub_df.iloc[0])
    height, width, layers = frame.shape
    # print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    os.makedirs(os.path.join(data_dir, f'videos_{len(cameras)}cams_concat'), exist_ok=True)
    video_name = os.path.join(data_dir, f'videos_{len(cameras)}cams_concat/seq{i}.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 1, (width*2,height*2), isColor=True)

    H = int(len(cameras)**0.5)
    W = len(cameras) // H
    for ii in tqdm(range(min_len)):
        try:
            canvas = np.zeros((height*H, width*W, 3), dtype=np.uint8)
            for ci in range(len(cam_dfs)):
                frame = read_raw_image_by_row(data_dir, cam_dfs[ci].iloc[ii])
                # frame = cv2.putText(frame, f'{cam_dfs[ci].iloc[ii].collected_on}', 
                #         (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                fi, fj = ci // W, ci % W
                canvas[height*fi:height*(fi+1), width*fj:width*(fj+1)] = frame
            video.write(canvas)
        except:
            print(f'{ii}th image read failed')

    # cv2.destroyAllWindows()
    video.release()
    break

  0%|          | 0/24 [00:00<?, ?it/s]

['T02', 'T06', 'T14', 'T10']
0 [24, 24, 24, 24]


100%|██████████| 24/24 [00:07<00:00,  3.15it/s]


### Create video from saved pngs

In [ ]:
def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    cv2.destroyAllWindows()
    video.release()

In [10]:
data_dir = '/data/jupiter/li.yu/data/Jupiter_human_on_path_3_fn_sequence'
# csv_file = '/data/jupiter/datasets/Jupiter_gilroy_reverse_manny_v2/master_annotations_1fps.csv'
# csv_file = '/data/jupiter/li.yu/data/mannequin_in_dust_v1/master_annotations.csv'
csv_file = f'{data_dir}/master_annotations.csv'
master_df = pd.read_csv(csv_file)
master_df = master_df.sort_values('collected_on')

# pred_dir = '/mnt/sandbox1/rakhil.immidisetti/output/driveable_terrain_model/471_cloud_v45_cutnpaste_s35/Jupiter_gilroy_reverse_manny_v2reverse_val_bestmodel/'
# pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v471_nobadiqnohard_6cls_cnp_alpha2_0805/Jupiter_gilroy_reverse_manny_v2/'
# pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1'
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence'
save_dir = pred_dir

# # save as a single video
# print(master_df.shape)
# video_name = os.path.join(save_dir, 'pred.mp4')
# ids = master_df.image_id.to_list()
# create_video(ids, pred_dir, video_name, fps=3)

# break into sequences
seq_dfs = get_sequences(master_df, interval=5, per_camera=True)
print(len(master_df), len(seq_dfs))
video_dir = os.path.join(pred_dir, 'videos')
os.makedirs(video_dir, exist_ok=True)
for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # create video
    # video_name = os.path.join(video_dir, str(si).zfill(3)+'.mp4')
    video_name = os.path.join(video_dir, f'{name}_{camera}_{si}.mp4')
    ids = seq_df.image_id.to_list()
    create_video(ids, pred_dir, video_name, fps=3)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


208 3
0 2021-11-10T00:33:05.931000 49
311 3141 3


100%|██████████| 49/49 [00:01<00:00, 37.81it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


total 49 used 49
1 2021-12-16T22:14:00.050000 71
311 3141 3


100%|██████████| 71/71 [00:01<00:00, 38.55it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


total 71 used 71
2 2022-11-10T23:19:10.901000 88
311 3141 3


100%|██████████| 88/88 [00:02<00:00, 40.61it/s]

total 88 used 88


### Create video from PP artifacts

In [7]:
def read_from_pp_artifacts(data_dir, df_row):
    data_path = os.path.join(data_dir, 'processed/images', df_row.id, 'stereo_output.npz')
    img = np.load(data_path)['left']
    img_norm = normalize_image(img, df_row.hdr_mode if 'hdr_mode' in df_row else True)
    return (img_norm * 255).astype(np.uint8)

def add_text(frame, txt_row):
    frame = cv2.putText(frame, f'Collected on: {txt_row.collected_on}', 
                        (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

In [19]:
data_root_dir = '/data/jupiter/li.yu/data'
# dataset = 'mannequin_in_dust_v1'
dataset = 'Jupiter_human_on_path_3_fn_sequence'
data_dir = os.path.join(data_root_dir, dataset)
video_dir = os.path.join(data_root_dir, dataset, 'videos')
sequence_dir = os.path.join(data_root_dir, dataset, 'sequences')
os.makedirs(video_dir, exist_ok=True)
os.makedirs(sequence_dir, exist_ok=True)

df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
seq_dfs = get_sequences(df, interval=5, per_camera=True)
print(df.shape, len(seq_dfs))

(208, 153) 3


In [9]:
height, width = 512, 1024
for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # create video
    video_name = os.path.join(video_dir, f'{name}_{camera}_{si}.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    for fi, df_row in seq_df.iterrows():
        frame = read_from_pp_artifacts(data_dir, df_row)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        video.write(frame)
    cv2.destroyAllWindows()
    video.release()

    # # create sequence
    # seq_dir_name = os.path.join(sequence_dir, f'{name}_{camera}_{si}')
    # os.makedirs(seq_dir_name, exist_ok=True)
    # fi = 0
    # for _, df_row in seq_df.iterrows():
    #     frame = read_from_pp_artifacts(data_dir, df_row)
    #     frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    #     cv2.imwrite(os.path.join(seq_dir_name, str(fi).zfill(3)+'_'+df_row.id+'.png'), frame)
    #     fi += 1
    # # break


0 2021-11-10T00:33:05.931000 49
1 2021-12-16T22:14:00.050000 71
2 2022-11-10T23:19:10.901000 88


### Read frame and add prediction results

In [20]:
# compare BRT model pred and CenterTrack pred
# pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1/output.csv'
pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence/output.csv'
pred_df = pd.read_csv(pred_csv)
print(pred_df.shape)

(208, 11)


In [11]:
video_with_pred_dir = os.path.join(data_root_dir, dataset, 'videos_with_pred')
# pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/2023-07-08T01:37:09.798000_front-center_15'
pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/brt50000/nopreimg_noprehm/2023-07-08T01:37:09.798000_front-center_15'
os.makedirs(video_with_pred_dir, exist_ok=True)
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 15:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # create video
    video_name = os.path.join(video_with_pred_dir, f'{name}_{camera}_{si}_nopreimg_noprehm.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    fi = 0
    for _, df_row in seq_df.iterrows():
        frame = cv2.imread(os.path.join(pred_sequence_dir, str(fi).zfill(3)+'_'+df_row.id+'.png'))
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
        fi += 1
    cv2.destroyAllWindows()
    video.release()

    # break


15 2023-07-08T01:37:09.798000 153


### read from video and prediction results to each frame

In [23]:
video_dir = '/home/li.yu/code/CenterTrack/results/'
old_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg.mp4'
new_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg_withbrtpred.mp4'
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 2:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # read video
    cam = cv2.VideoCapture(os.path.join(video_dir, old_video_name))

    # create video
    video_name = os.path.join(video_dir, new_video_name)
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    for _, df_row in seq_df.iterrows():
        _, frame = cam.read()
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
    cv2.destroyAllWindows()
    video.release()

    # break


2 2022-11-10T23:19:10.901000 88
